In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'depth_sheets'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=4719

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

# #remove duplicates
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']]

['This is a fraction: 100.02']

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values



['13 to 17 Inches', '18 Inches and Higher', '3 to 12 Inches', 'n/a']

In [7]:
#--------------------------------------------------------------------------------------------
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df=df[df['matches'].astype(str)!='[]']
# matches=df
# matches['matches'][0:20]

None


In [8]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
print(len(df))
regex_pattern=r'(?i)(?i)(\d+.?\s?deep)|(\d+\\.?\s?deep)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

2432


# na

In [9]:
df_na=df[df['matches'].astype(str)=='[]']
df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]
df_na

product_id external_id  \
1     1337725764    69748096   
2     1230264686    69723891   
4     1205703248    69541352   
6      817625884    69334580   
8      817652037    67362158   
...          ...         ...   
2425   817351461    67168965   
2426  1249190872    69712382   
2429   817424033    69551413   
2430   817424032    69551411   
2431   817641435    69534970   

                                                                    product_name  \
1       London Fog® Herringbone Flannel 3-Piece King Duvet Set in White and Grey   
2            Brielle Home Mabel 2-Piece Twin/Twin XL Comforter Set in Stone Blue   
4        Madison Park Quebec 3-Piece Reversible Queen Bedspread Set in Dark Grey   
6                               Ellen Tracy Grace King 3-Piece Quilt Set in Blue   
8          UGG® Devon 3-Piece Reversible Full/Queen Comforter Set in Sunset Pink   
...                                                                          ...   
2425                                           Polka Dot Twin Sheet Set in Lilac   
2426                              Devon Comforter Set / Twin/TwinXL / Deep Night   
2429  The Peanutshell™ Fitted Crib Sheets in Pink Elephants/Pink Hearts (2-Pack)   
2430        The Peanutshell™ Fitted Crib Sheets in Safari/Black Chevron (2-Pack)   
2431                        Disney® Lion King Photo Op Fitted Crib Sheet in Gold   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
1                                                                                                                                                                                                                               The London Fog Herringbone Flannel bed will keep you warm and comfortable and remind you of your favorite flannel shirt on a crisp fall or winter night. The herringbone pattern adds sophistication while keeping them easy to coordinate. This soft flannel bed from London Fog will keep you warm and comfortable and remind you of your favorite flannel shirt on a crisp fall or winter night The herringbone pattern adds sophistication while keeping them easy to coordinate Face and reverse fabric are made of extra soft 100% cotton flannel Twin Duvet Set includes: 90\" L x 68\" W duvet 20\" L x 26\" W sham Full / Queen Duvet Set includes: 90\" L x 90\" W duvet Two 20\" L x 26\" W sham King Duvet Set includes: 90\" L x 104\" W duvet Two 20\" L x 36\" W sham 100% Cotton Comforter insert not included Machine wash Imported   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                Transform your bedroom into a plush, private oasis with the Mable T

In [24]:
matches_na=df_na[['external_id',curation_col]]
matches_na

external_id Q:depth_sheets
1       69748096            n/a
2       69723891            n/a
4       69541352            n/a
6       69334580            n/a
8       67362158            n/a
...          ...            ...
2425    67168965            n/a
2426    69712382            n/a
2429    69551413            n/a
2430    69551411            n/a
2431    69534970            n/a

[1970 rows x 2 columns]

In [10]:
print(len(df[df['matches'].astype(str)!='[]']))
deep=df[df['matches'].astype(str)!='[]'][0:500]

462


In [11]:
deep['matches']= deep['matches'].apply(lambda x: re.sub(r'(?i)\\" deep|"deep|”\s?deep','',str(x))).apply(lambda x: re.sub(r'\\" deep|"deep|\"xa0deep|xa0deep|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

In [12]:
regex_pattern_comma=r','
deep['comma_matches'] = deep['matches'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
final_deep=deep[deep['comma_matches'].astype(str)=='[]']
final_deep.head()

product_id external_id  \
3   1311800965    63072075   
7    817595117    67526215   
12   817341615    60745545   
13  1329675400    64986838   
16   817539910    63334760   

                                                                                 product_name  \
3                                  Urban Habitat Heathered Jersey Twin XL Sheet Set in Purple   
7                                    ED Ellen DeGeneres Love Solid King Sheet Set in Charcoal   
12                                           True North Solid Flannel Queen Sheet Set in Blue   
13                                                          Quatrefoil Full Sheet Set in Navy   
16  Heartland® HomeGrown™ 325-Thread-Count Cotton Percale Twin XL Fitted Sheet in Light Green   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
3                                                                                                                                                                                                                                                                                                                                                                                                              Dress your bed with in cozy comfort with the Heathered Jersey Sheet Set from Urban Habitat. Boasting soft and smooth cotton construction, and available in a variety of versatile hues, this sheet set seamlessly blends with any bedding ensemble. 100% cotton jersey for comfort Machine wash Fully elasticized fitted sheet fits mattresses up to 15\" deep Solid, heathered tones offer a refreshing, stylish update to any bedding ensemble Open closure lets you slip pillows in and out easily Twin XL sheet set includes: One 66\" W x 96\" L Twin XL flat sheet One 39\" W x 80\" L Twin XL fitted sheet One 20\" W x 30\" L Standard/Queen pillowcase Imported Each Urban Habitat Heathered Jersey Sheet Set is sold separately   
7                                                                                                                                                      Drift off to sweet dreams with the casual style of the ED Ellen DeGeneres Love Solid Sheet Set. With an ultra-soft and lustrous feel, in an array of fashionable colors, the luxurious cotton blend sheets add a clean, tailored look to any bed. 48% cotton/42% Lyocell/10% linen Machine wash Solid tones bring a refreshing, luxurious update to any bedding ensemble Twin sheet set includes: 66\" W x 96\" L flat sheet 39\" W x 75\" L fitted sheet 20\" W x 30\" L standard pillowcase Full sheet set includes: 81\" W x 96\" L flat sheet 54\" W x 75\" L fitted sheet Two 20\" W x 30\" L standard pillowcases Queen sheet set includes: 90\" W x 102\" L flat sheet 60\" W x 80\" L fitted sheet Two 20\" W x 30\" L standard pillowcases King sheet set includes: 108\" W x 102\" L flat sheet 78\" W x 80\" L fitted sheet Two 20\" W x 40\" L king pillowcases Fitted sheet 

In [13]:
# deep['matches'].apply(lambda x: re.sub(r'in| ','',str(x))).apply(lambda x:float(x))
# deep['matches'].apply(lambda x: f'"{x}"').apply(lambda x: re.sub(r',','","',str(x))).apply(lambda x: re.sub(r'," ',',"',str(x)))

In [14]:
final_deep[curation_col]=final_deep['matches']
matches=final_deep[['external_id','matches']]
matches['matches']=matches['matches'].apply(lambda x: float(x))
matches.head()

external_id  matches
3     63072075     15.0
7     67526215     15.0
12    60745545     14.0
13    64986838     16.0
16    63334760     15.0

In [15]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['13 to 17 Inches', '18 Inches and Higher', '3 to 12 Inches', 'n/a']

In [16]:
#--------------------------------------------------------------------------------------------
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df=df[df['matches'].astype(str)!='[]']
# matches=df
# matches['matches'][0:20]

None


In [17]:
metric=" Inches"
range_string =""""
13 - 17 
Greater than 18
3 - 12
"""
# range_string =""""

#  Greater than 108
#  12 - 23.9
#  24 - 35.9
#  36 - 47.9
#  48 - 59.9
#  60 - 71.9
#  72 - 83.9
#  84 - 95.9
#  96 - 107.9
#  0 - 11.9
#  """

In [18]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower} to {upper}{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        no_words=original_val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return f'{no_words}{metric} and Higher'
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >= 13 and num <= 17 :
        return f'13 to 17  Inches'
        
    if num >= 18:
        return f'18 Inches and Higher'
        
    if num >= 3 and num <= 12:
        return f'3 to 12 Inches'
        
    else:
        return r"n/a"



In [19]:
matches[curation_col]=matches['matches'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches.head()

external_id  matches    Q:depth_sheets
3     63072075     15.0  13 to 17  Inches
7     67526215     15.0  13 to 17  Inches
12    60745545     14.0  13 to 17  Inches
13    64986838     16.0  13 to 17  Inches
16    63334760     15.0  13 to 17  Inches

In [21]:
matches=matches[matches[curation_col].astype(str)!='n/a']
matchez=matches[['external_id',curation_col]]
matchez

external_id        Q:depth_sheets
3       63072075      13 to 17  Inches
7       67526215      13 to 17  Inches
12      60745545      13 to 17  Inches
13      64986838      13 to 17  Inches
16      63334760      13 to 17  Inches
24      69704496        3 to 12 Inches
28      69591022        3 to 12 Inches
36      65264027        3 to 12 Inches
53      69759121      13 to 17  Inches
58      69749028      13 to 17  Inches
63      69684631        3 to 12 Inches
86      60452603      13 to 17  Inches
92      63127058      13 to 17  Inches
93      62317948  18 Inches and Higher
95      47683822  18 Inches and Higher
104     69691971      13 to 17  Inches
105     47677456  18 Inches and Higher
106     61832961      13 to 17  Inches
115     63859782      13 to 17  Inches
126     69750010        3 to 12 Inches
138     69722264      13 to 17  Inches
150     67526277      13 to 17  Inches
152     64310480      13 to 17  Inches
159     69704495        3 to 12 Inches
175     69528626        3 to 12 Inches
178     68422493      13 to 17  Inches
180     67919611        3 to 12 Inches
187     65192481        3 to 12 Inches
189     61561762        3 to 12 Inches
190     65192467        3 to 12 Inches
213     69759131      13 to 17  Inches
218     68422578      13 to 17  Inches
220     67002269        3 to 12 Inches
236     69530758        3 to 12 Inches
240     62317887  18 Inches and Higher
242     68262006  18 Inches and Higher
243     68262013  18 Inches and Higher
247     47736627      13 to 17  Inches
254     62368803  18 Inches and Higher
265     47736535      13 to 17  Inches
286     61239432      13 to 17  Inches
287     61239463      13 to 17  Inches
289     47677517  18 Inches and Higher
293     69591021        3 to 12 Inches
307     65264850        3 to 12 Inches
314     69750009        3 to 12 Inches
318     68847111        3 to 12 Inches
327     68455590        3 to 12 Inches
328     69528700        3 to 12 Inches
329     69691967      13 to 17  Inches
340     67103898        3 to 12 Inches
342     62368780  18 Inches and Higher
348     69701453        3 to 12 Inches
356     45928642      13 to 17  Inches
364     67462599      13 to 17  Inches
370     66851417        3 to 12 Inches
382     64133393      13 to 17  Inches
384     62335775      13 to 17  Inches
398     45737572  18 Inches and Higher
400     67002252        3 to 12 Inches
402     60745378      13 to 17  Inches
403     60745453      13 to 17  Inches
404     63161090  18 Inches and Higher
410     69591007        3 to 12 Inches
415     66513230        3 to 12 Inches
440     69652005      13 to 17  Inches
447     69298080        3 to 12 Inches
457     68846961        3 to 12 Inches
463     62380959        3 to 12 Inches
480     69691813        3 to 12 Inches
490     63614824      13 to 17  Inches
504     69587559        3 to 12 Inches
508     68691400        3 to 12 Inches
510     69591012        3 to 12 Inches
512     66989479        3 to 12 Inches
534     69490223        3 to 12 Inches
546     67002283        3 to 12 Inches
553     64997520        3 to 12 Inches
556     64997537        3 to 12 Inches
563     69684962        3 to 12 Inches
565     45723605        3 to 12 Inches
580     64989419      13 to 17  Inches
584     67462551      13 to 17  Inches
592     60488732  18 Inches and Higher
593     47683686  18 Inches and Higher
595     69686166        3 to 12 Inches
606     69730497        3 to 12 Inches
623     69595490        3 to 12 Inches
636     68852092        3 to 12 Inches
637     68847098        3 to 12 Inches
639     67103843        3 to 12 Inches
643     65098974        3 to 12 Inches
652     62317931  18 Inches and Higher
654     62380645        3 to 12 Inches
663     62379410        3 to 12 Inches
664     63859775      13 to 17  Inches
666     68129354      13 to 17  Inches
668     69547814        3 to 12 Inches
674     69206085      13 to 17  Inches
675     62318211  18 Inches and Higher
681     63614831      13 to 17  Inches
683     61020207  18 Inches

# send to the folder for upload

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [26]:
looks_good('Bed Bath & Beyond', attribute, df, matches_na)